# Recomendação por filtragem colaborativa de Usuário

## Setup

In [2]:
import pandas as pd

dados_formados_path = 'input/dados_formados.csv'
dados_regulares_path = 'input/dados_regulares.csv'
dados_trancados_path = 'input/dados_trancados.csv'
dados_desistentes_path = 'input/dados_desistentes.csv'

dados = {
    "formados": pd.read_csv(dados_formados_path, delimiter=';'),
    "regulares": pd.read_csv(dados_regulares_path, delimiter=';'),
    "desistentes": pd.read_csv(dados_desistentes_path, delimiter=';'),
    "trancados": pd.read_csv(dados_trancados_path, delimiter=';')
}

dados.update(todos = pd.concat([dados["desistentes"], dados["regulares"], dados["formados"], dados["trancados"]]))


## Funções auxiliares
### select_random_student(data : pd.DataFrame, max_tries : int, minimum_entry_year : int) -> int
Seleciona um ID de estudante aleatório.

### get_all_data_from_student(student_id : int, data : pd.DataFrame) -> pd.DataFrame
Retorna apenas os dados do estudante presentes no dataframe data.

### get_user_array(student_id : int, data : pd.DataFrame) -> defaultdict
Retorna um defaultdict com as disciplinas e menor nota obtida que o aluno cursou.

### sugestoes_populares(student_id : int, max_sugestoes : int, data : pd.DataFrame) -> List
Sugere disciplinas para o estudante com base nas disciplinas entre os alunos formados até 2023

### todas_disciplinas: np.array
Array com o codigo de todas as disciplinas dentro dos dados

In [19]:
def select_random_student(data : pd.DataFrame, max_tries : int, minimum_entry_year : int) -> int:
    id = 0
    tries = 0

    minimum_entry_year -= 2000
    
    while id == 0 and tries < max_tries:
        try:
            id = data[[int(x[3].split('/')[2].split(' ')[0]) >= minimum_entry_year for x in data.values]].sample()
            return int(id.ID_ANONIMO.iloc[0])
        except:
            id = 0
            tries += 1
    return 0

In [4]:
def get_all_data_from_student(student_id : int, data : pd.DataFrame):
    return data[[x == student_id for x in data.ID_ANONIMO]]
    pass

In [5]:
from typing import List
from collections import defaultdict

def centralized_cosine(user_array):
    vec_med = sum(user_array.values())/len(user_array.values())
    for key in user_array:
        user_array[key] -= vec_med
    return user_array

def get_user_array(student_id : int, data : pd.DataFrame, centralize_cosine = True):
    student_data = get_all_data_from_student(student_id, data)

    student_array : Dict[str, float] = defaultdict(lambda : 0)
    
    for course in student_data.values:
        if student_array[course[4]] == 0:
            #print([course[4], float(str(course[7]).replace(',','.')),(course[5] - 2000)*2 + course[6]])
            student_array[course[4]] = float(str(course[7]).replace(',','.'))
            if student_array[course[4]] != student_array[course[4]]:
                student_array[course[4]] = 0
        else:
            nota = float(str(course[7]).replace(',','.'))
            student_array[course[4]] = nota if nota < student_array[course[4]] else student_array[course[4]]
    
    if centralize_cosine and len(student_array) != 0:
        student_array = centralized_cosine(student_array)
    
    return student_array

In [6]:
import numpy as np

disciplinas = []
with open('input/dependencias.txt') as f:
    for line in f:
        curr = line.split(';')
        if 'P8' in curr:
            curr = curr[1].split('\n')[0]
            disciplinas.append(curr)

disciplinas_obrigatorias = ['GE70D', 'EEC31', 'CSS30','EEX23']
optativas = np.array(disciplinas)
optativas = optativas[[disc not in disciplinas_obrigatorias for disc in optativas]]

In [7]:
# Dados dos formados contendo apenas optativas, do melhor jeito que conseguimos (sem certeza de completude ou corretude)

dados_formados_optativas = dados["formados"][[int(x[3].split('/')[2].split(' ')[0]) >= 14 for x in dados["formados"].values]]
dados_formados_optativas = dados_formados_optativas[[float(str(x[7]).replace(',','.')) >= 6 for x in dados_formados_optativas.values]]
dados_formados_optativas = dados_formados_optativas[[x[4] != 'ES70N' or x[5] > 2017 for x in dados_formados_optativas.values]]
dados_formados_optativas = dados_formados_optativas[[x[4] != 'FI70D' or x[5] > 2017 for x in dados_formados_optativas.values]]
dados_formados_optativas = dados_formados_optativas[[x[4] != 'FI70A' or x[5] > 2017 for x in dados_formados_optativas.values]]
dados_formados_optativas = dados_formados_optativas[[x[4] != 'GE70F' or x[5] > 2017 for x in dados_formados_optativas.values]]
dados_formados_optativas = dados_formados_optativas[dados_formados_optativas.CODIGO.isin(disciplinas)]

In [8]:
from collections import Counter

popular_courses = Counter([materia for materia in dados_formados_optativas["CODIGO"]])

In [9]:
# Recomendação de disciplinas populares
from typing import List

# Recebe um estudante e sugere as disciplinas mais populares que ele não fez ainda
def sugestoes_populares(student_id : int, max_sugestoes : int, data : pd.DataFrame) -> List:
    suggestions = [interest for interest, _ in popular_courses.most_common() if interest not in get_all_data_from_student(student_id, data)["CODIGO"].unique()]
    return suggestions[:max_sugestoes]

In [10]:
todas_disciplinas = []

for frame in dados:
    todas_disciplinas.extend(dados[frame]["CODIGO"].unique())

todas_disciplinas = np.unique(np.array(todas_disciplinas))

## Recomendação por Filtragem Colaborativa de Usuários

In [11]:
from numpy import dot
from numpy.linalg import norm
import math

def cos_sim(a : List[int], b : List[int]) -> int:
    return dot(a, np.transpose(b))/(norm(a)*norm(b))

# O cache deve ser criado com os mesmos dados utilizados nas funções
def create_cache_user_arrays(data, centralize_cosine = True):
    return {int(identifier): get_user_array(identifier, data, centralize_cosine = centralize_cosine) for identifier in data["ID_ANONIMO"].unique() if identifier == identifier}
        
def most_similar_students_to(user_id : int, student_ids : List[int], centralize_cosine = True, include_self = False,
                            cache = None):
    pairs = []

    user_array = get_user_array(user_id, dados["todos"], centralize_cosine = centralize_cosine)
    user_list = []
    for course in user_array:
        user_list.append(user_array[course] if user_array[course] > 0 else 0)

    for id in student_ids:
        if include_self or id != user_id:
            if cache != None:
                student_array = cache[id]
            else:
                student_array = get_user_array(id, dados["todos"], centralize_cosine = centralize_cosine)
            
            student_list = []

            for course in user_array:
                #print(student_array[course] if student_array[course] > 0 else 1)
                student_list.append(student_array[course] if student_array[course] > 0 else 0)


            if norm(user_list)*norm(student_list) != 0 and cos_sim(user_list, student_list) == cos_sim(user_list, student_list):
                pairs.append((int(id), cos_sim(user_list, student_list)))
    
    return sorted(pairs, key = lambda pair: pair[-1], reverse = True)

In [12]:
def user_based_suggestions(user_id: int, include_user_interests: bool = False, 
                           only_optionals = True, data_rec = dados["formados"], data_user = dados["todos"],
                           disciplinas_optativas = optativas, centralize_cosine = True, reduce_popularity_weight = True, cache = None):
    # Some as semelhanças
    suggestions : Dict[str, float] = defaultdict(float)
    times_coursed : Dict[str, float] = defaultdict(int)
        
    if cache != None:
        for other_user_id, similarity in most_similar_students_to(user_id, data_rec["ID_ANONIMO"].unique(), centralize_cosine = centralize_cosine, cache = cache):
            for interest in get_all_data_from_student(other_user_id, data_rec)["CODIGO"].values:
                suggestions[interest] += similarity
                times_coursed[interest] += 1
    else:
        for other_user_id, similarity in most_similar_students_to(user_id, data_rec["ID_ANONIMO"].unique(), centralize_cosine = centralize_cosine):
            for interest in get_all_data_from_student(other_user_id, data_rec)["CODIGO"].values:
                suggestions[interest] += similarity
                times_coursed[interest] += 1
    
    if reduce_popularity_weight:
        for codigo in suggestions:
            suggestions[codigo] = suggestions[codigo]/times_coursed[codigo]
    
    # Converta em uma lista classificada 
    suggestions = sorted(suggestions.items(),
                        key = lambda pair: pair[-1],
                        reverse = True)

    # Exclua nao optativas
    if only_optionals:
        suggestions = [(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestion in disciplinas_optativas]
    
    # Exclua interesses existentes
    if include_user_interests:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestion not in get_all_data_from_student(user_id, data_user)["CODIGO"].values]

In [13]:
cache = create_cache_user_arrays(dados["todos"], centralize_cosine = True)

In [46]:
def trim_user_data(user_data, num_materias):
    trimmed_data = user_data.copy()

    trimmed_data = trimmed_data.sort_values(by=['ANO','PERIODO.1'])
    # Dropping last n rows using drop
    trimmed_data.drop(trimmed_data.tail(num_materias).index,
            inplace = True)

    return trimmed_data


In [47]:
user_id = select_random_student(dados['formados'], 20, 2016)
print(user_id)

190493


In [48]:
user_data = get_all_data_from_student(user_id, dados['formados'])
trimmed_data = trim_user_data(user_data, 12)

In [49]:
user_data = user_data.sort_values(by=['ANO','PERIODO.1'])
user_data.to_csv(f'output/dados_{user_id}.csv', sep=',')

In [50]:
trimmed_data = trimmed_data.sort_values(by=['ANO','PERIODO.1'])
trimmed_data.to_csv(f'output/dados_trimmed_{user_id}.csv', sep=',')


In [1]:
user_id = 202266

dados_user = get_all_data_from_student(user_id, dados["todos"])

with open('sugestoes.txt', 'w') as file:
    i = 1
    file.write(f'{user_id}\n')
    for item in user_based_suggestions(user_id, data_user = dados["regulares"], data_rec = dados["formados"], include_user_interests = True, cache= cache):
        #print('Sugestao ' + str(i) + ': ' + item[0] + ', peso: ' + format(item[1], '.2f'))
        i += 1
        file.write(f'{item[0]}\n')
        

NameError: name 'get_all_data_from_student' is not defined

In [14]:
dados["regulares"]

,ID_ANONIMO,CR,PERIODO,INGRESSO,CODIGO,ANO,PERIODO.1,NOTA,CH,SITUACAOALUNO
0,11357,"0,6967",5,23/01/23,CSD21,2023,1,"4,7",45,Regular
1,11357,"0,6967",5,23/01/23,ES70P,2023,1,"7,6",45,Regular
2,11357,"0,6967",5,23/01/23,CSE20,2023,1,6,60,Regular
3,11357,"0,6967",5,23/01/23,ES70G,2023,1,"7,3",45,Regular
4,11357,"0,6967",5,23/01/23,CSF20,2023,1,"6,3",45,Regular
...,...,...,...,...,...,...,...,...,...,...
19877,261036,"0,695",2,09/08/23,ICSF13,2023,2,"6,6",90,Regular
19878,261911,"0,7143",1,17/08/23,ICSD21,2023,2,8,45,Regular
19879,261911,"0,7143",1,17/08/23,ELEX10,2023,2,8,45,Regular
19880,261911,"0,7143",1,17/08/23,ICSF13,2023,2,6,90,Regular


# Testes

### Recall@K

Recall@K = number of relevant items in the top K / total number of relevant items

### Precision@K

Precision@K = number of relevant items in the top K / K

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dados["formados"]["ID_ANONIMO"].unique(), test_size=0.3)

dados.update(train = dados["todos"].loc[dados["todos"]['ID_ANONIMO'].isin(train)], 
             test = dados["todos"].loc[dados["todos"]['ID_ANONIMO'].isin(test)])

In [ ]:
from tqdm import tqdm

# Para cada usuario em test, vemos quantas disciplinas que ele cursou estão entre as 10 mais sugeridas. 
# No fim, tiramos a média por usuário.

def teste_sugestao(dados_train = dados["train"], dados_test = dados["test"], cache = None, K = 10, centralize_cosine = True, reduce_popularity_weight = True):
    cache = create_cache_user_arrays(pd.concat([dados["train"], dados["test"]]), centralize_cosine = True)
    
    media_p = 0
    maximo_p = 0
    minimo_p = 9999
    res_p = 0

    media_r = 0
    maximo_r = 0
    minimo_r = 9999
    res_r = 0
    
    n_users = 0

    data = dados_test["ID_ANONIMO"].unique()

    for i in tqdm(range(len(data))):
        user = data[i]

        user_data = get_all_data_from_student(user, dados_test)

        user_data_filtered = user_data[[x[2] < 8 for x in user_data.values]]

        sugestoes = user_based_suggestions(202266, data_rec = dados_train, data_user=user_data_filtered, include_user_interests = True, cache=cache, 
                                           centralize_cosine = centralize_cosine, reduce_popularity_weight=reduce_popularity_weight)
        if(len(user_data[user_data.CODIGO.isin(optativas)]) > 4):
            #print(len(user_data[user_data.CODIGO.isin(optativas)]), " optativas")
            n_users += 1
            # Precision @ K
            res_p = sum([x[0] in user_data["CODIGO"].unique() for x in sugestoes[:K]])/K
            media_p += res_p
            if maximo_p < res_p:
                maximo_p = res_p
            if minimo_p > res_p:
                minimo_p = res_p
            # Recall @ K
            res_r = sum([x[0] in user_data["CODIGO"].unique() for x in sugestoes[:K]])/len(user_data[user_data.CODIGO.isin(optativas)])
            media_r += res_r
            if maximo_r < res_r:
                maximo_r = res_r
            if minimo_r > res_r:
                minimo_r = res_r
        #else:
            #print("Usuario ", user, " sem dados de optativas")
            
    print(n_users, " dados para teste")

    print("Media Precisão @", K ,"= ", media_p/len(dados_test["ID_ANONIMO"].unique()))
    print("Maximo Precisão @", K , "= ", maximo_p)
    print("Minimo Precisão @", K , "= ", minimo_p)

    print("Media Recall @", K ,"= ", media_r/len(dados_test["ID_ANONIMO"].unique()))
    print("Maximo Recall @", K , "= ", maximo_r)
    print("Minimo Recall @", K , "= ", minimo_r)
    pass

In [ ]:
#cache_test = create_cache_user_arrays(pd.concat([dados["train"], dados["test"]]), centralize_cosine = True)

In [ ]:
train, test = train_test_split(dados["formados"]["ID_ANONIMO"].unique(), test_size=0.3)

dados.update(train = dados["todos"].loc[dados["todos"]['ID_ANONIMO'].isin(train)], 
             test = dados["todos"].loc[dados["todos"]['ID_ANONIMO'].isin(test)])

teste_sugestao( K = 20, centralize_cosine=True, reduce_popularity_weight = True)

100%|██████████| 89/89 [00:41<00:00,  2.14it/s]

50  dados para teste
Media Precisão @ 20 =  0.058426966292134834
Maximo Precisão @ 20 =  0.25
Minimo Precisão @ 20 =  0.0
Media Recall @ 20 =  0.15878631356159442
Maximo Recall @ 20 =  0.8
Minimo Recall @ 20 =  0.0


In [ ]:

teste_sugestao( K = 20, centralize_cosine=False, reduce_popularity_weight = True)

100%|██████████| 89/89 [00:42<00:00,  2.09it/s]

50  dados para teste
Media Precisão @ 20 =  0.01685393258426967
Maximo Precisão @ 20 =  0.1
Minimo Precisão @ 20 =  0.0
Media Recall @ 20 =  0.05002647165568514
Maximo Recall @ 20 =  0.4
Minimo Recall @ 20 =  0.0


In [ ]:

teste_sugestao( K = 20, centralize_cosine=True, reduce_popularity_weight = False)

100%|██████████| 89/89 [00:47<00:00,  1.89it/s]

50  dados para teste
Media Precisão @ 20 =  0.13820224719101126
Maximo Precisão @ 20 =  0.45
Minimo Precisão @ 20 =  0.05
Media Recall @ 20 =  0.36589306448857023
Maximo Recall @ 20 =  1.0
Minimo Recall @ 20 =  0.16666666666666666


In [ ]:

teste_sugestao( K = 20, centralize_cosine=False, reduce_popularity_weight = False)

100%|██████████| 89/89 [00:48<00:00,  1.84it/s]

50  dados para teste
Media Precisão @ 20 =  0.13820224719101126
Maximo Precisão @ 20 =  0.45
Minimo Precisão @ 20 =  0.05
Media Recall @ 20 =  0.36589306448857023
Maximo Recall @ 20 =  1.0
Minimo Recall @ 20 =  0.16666666666666666


In [ ]:
train, test = train_test_split(dados["formados"]["ID_ANONIMO"].unique(), test_size=0.3)

dados.update(train = dados["todos"].loc[dados["todos"]['ID_ANONIMO'].isin(train)], 
             test = dados["todos"].loc[dados["todos"]['ID_ANONIMO'].isin(test)])

teste_sugestao( K = 15, centralize_cosine=True, reduce_popularity_weight = True)

100%|██████████| 89/89 [00:41<00:00,  2.14it/s]

50  dados para teste
Media Precisão @ 20 =  0.058426966292134834
Maximo Precisão @ 20 =  0.25
Minimo Precisão @ 20 =  0.0
Media Recall @ 20 =  0.15878631356159442
Maximo Recall @ 20 =  0.8
Minimo Recall @ 20 =  0.0


In [ ]:

teste_sugestao( K = 15, centralize_cosine=False, reduce_popularity_weight = True)

100%|██████████| 89/89 [00:42<00:00,  2.09it/s]

50  dados para teste
Media Precisão @ 20 =  0.01685393258426967
Maximo Precisão @ 20 =  0.1
Minimo Precisão @ 20 =  0.0
Media Recall @ 20 =  0.05002647165568514
Maximo Recall @ 20 =  0.4
Minimo Recall @ 20 =  0.0


In [ ]:

teste_sugestao( K = 15, centralize_cosine=True, reduce_popularity_weight = False)

100%|██████████| 89/89 [00:47<00:00,  1.89it/s]

50  dados para teste
Media Precisão @ 20 =  0.13820224719101126
Maximo Precisão @ 20 =  0.45
Minimo Precisão @ 20 =  0.05
Media Recall @ 20 =  0.36589306448857023
Maximo Recall @ 20 =  1.0
Minimo Recall @ 20 =  0.16666666666666666


In [ ]:

teste_sugestao( K = 15, centralize_cosine=False, reduce_popularity_weight = False)

100%|██████████| 89/89 [00:48<00:00,  1.84it/s]

50  dados para teste
Media Precisão @ 20 =  0.13820224719101126
Maximo Precisão @ 20 =  0.45
Minimo Precisão @ 20 =  0.05
Media Recall @ 20 =  0.36589306448857023
Maximo Recall @ 20 =  1.0
Minimo Recall @ 20 =  0.16666666666666666


In [ ]:
train, test = train_test_split(dados["formados"]["ID_ANONIMO"].unique(), test_size=0.3)

dados.update(train = dados["todos"].loc[dados["todos"]['ID_ANONIMO'].isin(train)], 
             test = dados["todos"].loc[dados["todos"]['ID_ANONIMO'].isin(test)])

teste_sugestao( K = 10, centralize_cosine=True, reduce_popularity_weight = True)

100%|██████████| 89/89 [00:41<00:00,  2.14it/s]

50  dados para teste
Media Precisão @ 20 =  0.058426966292134834
Maximo Precisão @ 20 =  0.25
Minimo Precisão @ 20 =  0.0
Media Recall @ 20 =  0.15878631356159442
Maximo Recall @ 20 =  0.8
Minimo Recall @ 20 =  0.0


In [ ]:

teste_sugestao( K = 10, centralize_cosine=False, reduce_popularity_weight = True)

100%|██████████| 89/89 [00:42<00:00,  2.09it/s]

50  dados para teste
Media Precisão @ 20 =  0.01685393258426967
Maximo Precisão @ 20 =  0.1
Minimo Precisão @ 20 =  0.0
Media Recall @ 20 =  0.05002647165568514
Maximo Recall @ 20 =  0.4
Minimo Recall @ 20 =  0.0


In [ ]:

teste_sugestao( K = 10, centralize_cosine=True, reduce_popularity_weight = False)

100%|██████████| 89/89 [00:47<00:00,  1.89it/s]

50  dados para teste
Media Precisão @ 20 =  0.13820224719101126
Maximo Precisão @ 20 =  0.45
Minimo Precisão @ 20 =  0.05
Media Recall @ 20 =  0.36589306448857023
Maximo Recall @ 20 =  1.0
Minimo Recall @ 20 =  0.16666666666666666


In [ ]:

teste_sugestao( K = 10, centralize_cosine=False, reduce_popularity_weight = False)

100%|██████████| 89/89 [00:48<00:00,  1.84it/s]

50  dados para teste
Media Precisão @ 20 =  0.13820224719101126
Maximo Precisão @ 20 =  0.45
Minimo Precisão @ 20 =  0.05
Media Recall @ 20 =  0.36589306448857023
Maximo Recall @ 20 =  1.0
Minimo Recall @ 20 =  0.16666666666666666
